# Improved Model

In this Jupyter Notebook you will find the code for the improved model implementation which increased the accuracy of the original model by 9%.

### Define which GPU I want to run the code on

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Import the relevant libraries

In [2]:
import numpy as np
import torch
from torchvision import models, transforms, datasets
import torch.nn.functional as F
from torch import nn, optim
import face_detector
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import time
from PIL import Image
from efficientnet_pytorch import EfficientNet

### Make sure the code is running on CUDA

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Working on device {device}')

Working on device cuda


In [5]:
class Network(nn.Module):
    
    '''Class which defines the classification layer which is going to be added
    to the end of the pretrained EfficientNet model.'''
    
    def __init__(self):
        super(Network, self).__init__()
        
        self.layer1 = nn.Linear(2560,5000)
        self.layer2 = nn.Linear(5000, 2500)
        self.layer3 = nn.Linear(2500, 1250)
        self.layer4 = nn.Linear(1250, 200)
        self.layer5 = nn.Linear(200, 2)

    def forward(self, x):
        
        #network 1
        x = x.view(x.shape[0], -1)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = F.log_softmax(self.layer5(x), dim=1)
        
        return x

In [6]:
class EnsembleNetwork(nn.Module):
    
    '''Class defining the Ensemble model using 5 pretrained 
    EfficientNet models.'''
    
    def __init__(self):
        
        super(EnsembleNetwork, self).__init__()
        
        self.model_1 = EfficientNet.from_pretrained('efficientnet-b7')
        self.model_2 = EfficientNet.from_pretrained('efficientnet-b7')
        self.model_3 = EfficientNet.from_pretrained('efficientnet-b7')
        self.model_4 = EfficientNet.from_pretrained('efficientnet-b7')
        self.model_5 = EfficientNet.from_pretrained('efficientnet-b7')
        
        self.model_1._fc = nn.Identity()
        self.model_2._fc = nn.Identity()
        self.model_3._fc = nn.Identity()
        self.model_4._fc = nn.Identity()
        self.model_5._fc = nn.Identity()
        
        for param in self.model_1.parameters():
            param.requires_grad = False
        for param in self.model_2.parameters():
            param.requires_grad = False
        for param in self.model_3.parameters():
            param.requires_grad = False
        for param in self.model_4.parameters():
            param.requires_grad = False
        for param in self.model_5.parameters():
            param.requires_grad = False
            
        self.model_1._fc = Network()
        self.model_2._fc = Network()
        self.model_3._fc = Network()
        self.model_4._fc = Network()
        self.model_5._fc = Network()

        
        self.layer1 = nn.Linear(10,2)
        
    def forward(self, x):
        
        x1 = self.model_1(x[0])
        x2 = self.model_2(x[1])
        x3 = self.model_3(x[2])
        x4 = self.model_4(x[3])
        x5 = self.model_5(x[4])
        
        output = torch.cat([x1, x2, x3, x4, x5], dim=1)
        x = F.log_softmax(self.layer1(output), dim=1)
        
        return x

### Batching the data ready for training

In this preprocessing step we perform the following actions:

- Transforms the data using torchvision.transforms
- Create an image data set by specifying the directory to read the data
- Create a loader which feeds batches of images to the ensemble model.

In [7]:
data_transforms = transforms.Compose([transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225])])

In [8]:
extensions = '.mp4'

data_dir = './images/'

In [9]:
train_data = datasets.ImageFolder(data_dir, transform=data_transforms)

In [10]:
trainloader = torch.utils.data.DataLoader(train_data,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=0)

In [11]:
ensemble_network = EnsembleNetwork()

Loaded pretrained weights for efficientnet-b7
Loaded pretrained weights for efficientnet-b7
Loaded pretrained weights for efficientnet-b7
Loaded pretrained weights for efficientnet-b7
Loaded pretrained weights for efficientnet-b7


EnsembleNetwork(
  (model_1): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          64, 16, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          16, 64, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
    

In [12]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(ensemble_network.parameters(), lr=1e-4)

In [13]:
def image_split(x):
    
    '''Splits a 224x224 image into 4 equal quarters.
    
    Returns: 
    Whole image, Top left corner, Bottom left corner,
    Top right corner, Bottom right corner.'''
    
    return x, x[:,:,0:112,0:112], x[:,:,0:112,112:224],\
           x[:,:,112:224,0:112], x[:,:,112:224,112:224]

In [14]:
def impurity(x):
    
    '''Function to calculate the impurity of the model output. This is used to 
    make sure the model does not output the same class for every observation in the
    batch.'''
    
    x = x.cpu().numpy()
    x = x.ravel()
    total = 1

    for i in np.unique(x):
        
        total -= np.mean(x == i) ** 2
        
    return total

In [15]:
ensemble_network.to(device)

In [16]:
epoch = 20

start = time.time()
for e in range(epoch):
    
    accuracy_list = []

    train_losses = []
    running_loss = 0

    for images, labels in trainloader:
        
        ensemble_network.train()
        images = images.to(device)
        labels = labels.to(device)
        splits = image_split(images)
        log_output = ensemble_network(splits)
        
        loss = criterion(log_output, labels)
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()
        train_losses.append(running_loss/len(trainloader))
        
    
        with torch.no_grad():
            ensemble_network.eval()
            preds = ensemble_network(splits)
            
            preds = torch.exp(preds)
        
            top_p, top_class = preds.topk(1, dim=1)
            
            equals = top_class == labels.view(*top_class.shape)

            accuracy_list.append(np.mean(equals.cpu().numpy()))

    print(f"Training loss: {running_loss} | Epoch Accuracy : {np.mean(accuracy_list)} | pred_impurity : {impurity(top_class)}")

end = time.time()

print(f'Time taken for {epoch} epochs : {end - start}')

Training loss: 8.620929569005966 | Epoch Accuracy : 0.4073660714285714 | pred_impurity : 0.0
Training loss: 6.17022117972374 | Epoch Accuracy : 0.8206845238095238 | pred_impurity : 0.0
Training loss: 5.520606577396393 | Epoch Accuracy : 0.8227306547619048 | pred_impurity : 0.0
Training loss: 5.030160367488861 | Epoch Accuracy : 0.8236607142857143 | pred_impurity : 0.0
Training loss: 4.823809593915939 | Epoch Accuracy : 0.8240327380952381 | pred_impurity : 0.0
Training loss: 4.607718288898468 | Epoch Accuracy : 0.8234747023809524 | pred_impurity : 0.0
Training loss: 4.421588480472565 | Epoch Accuracy : 0.8247767857142857 | pred_impurity : 0.0
Training loss: 4.1183750331401825 | Epoch Accuracy : 0.8279389880952381 | pred_impurity : 0.03507653061224496
Training loss: 3.9813032150268555 | Epoch Accuracy : 0.8422619047619048 | pred_impurity : 0.03507653061224496
Training loss: 3.905114457011223 | Epoch Accuracy : 0.8536086309523809 | pred_impurity : 0.16262755102040824
Training loss: 3.4255